<a href="https://colab.research.google.com/github/geraschenko/CME241/blob/main/assignments_raw/assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stanford CME 241 (Winter 2024) - Assignment 3

**Due: Jan 29 @ 11:59pm Pacific Time on Gradescope.**

Assignment instructions:
- **Please solve questions 1 and 2, and choose one of questions 3 or 4.**
- Empty code blocks are for your use. Feel free to create more under each section as needed.

Submission instructions:
- When complete, fill out your publicly available GitHub repo file URL below, then export or print this .ipynb file to PDF and upload the PDF to Gradescope.

*Link to this ipynb file in your public GitHub repo (replace below URL with yours):*

https://github.com/geraschenko/CME241/blob/main/assignments/assignment3.ipynb

## Imports

In [4]:
!pip install -q git+https://github.com/TikhonJelvis/RL-book.git

  Preparing metadata (setup.py) ... done


In [56]:
import rl.dynamic_programming
from rl.distribution import (
    Distribution, Categorical, Choose, Constant, FiniteDistribution
)
from rl.markov_process import (
    State, Terminal, NonTerminal, FiniteMarkovProcess, FiniteMarkovRewardProcess
)
from rl.policy import FinitePolicy, FiniteDeterministicPolicy
from rl.markov_decision_process import FiniteMarkovDecisionProcess
import dataclasses
from typing import Callable, Iterator, Mapping, Optional, TypeVar
import itertools
import collections
import numpy as np
import numpy.typing as npt
import scipy
import matplotlib.pyplot as plt

## Question 1
**Analytic Optimal Actions and Cost.**
Consider a continuous-states, continuous-actions, discrete-time, non-terminating MDP with state space as $\mathbb{R}$ and action space as $\mathbb{R}$. When in state $s\in \mathbb{R}$, upon taking action $a\in \mathbb{R}$, one transitions to next state $s' \in \mathbb{R}$ according to a normal distribution $s' \sim \mathcal{N}(s, \sigma^2)$ for a fixed variance $\sigma^2 \in \mathbb{R}^+$. The corresponding cost associated with this transition is $e^{as'}$, i.e., the cost depends on the action $a$ and the state $s'$ one transitions to. The problem is to minimize the infinite-horizon {\em Expected Discounted-Sum of Costs} (with discount factor $\gamma < 1$). For this assignment, solve this problem just for the special case of $\gamma = 0$ (i.e., the myopic case) using elementary calculus. Derive an analytic expression for the optimal action in any state and the corresponding optimal cost.


Since $\gamma = 0$, the return is equal to the reward, $-e^{as'}$, where $s'\sim \mathcal N(s, \sigma^2)$. The expected total cost (i.e. negative expected return) for action $a$ is therefore
$$\begin{align*}
\frac{1}{\sigma\sqrt{2\pi}}\int e^{as'}e^{-\frac{(s' - s)^2}{2\sigma^2}}ds'
 &= \frac{1}{\sigma\sqrt{2\pi}} \int \exp\Bigl(-\frac{1}{2}\frac{s'^2}{\sigma^2} + s'\bigl(\frac{s}{\sigma^2} + a\bigr) - \frac{1}{2}\frac{s^2}{\sigma^2}\Bigr)ds' \\
 &= \exp\Bigl(- \frac{1}{2}\frac{s^2}{\sigma^2} + \frac{1}{2}\frac{(s+\sigma^2a)^2}{\sigma^2}\Bigr) \cdot \underbrace{\frac{1}{\sigma\sqrt{2\pi}}\int \exp\Bigl(-\frac 1{2\sigma^2}(s' - (s+\sigma^2a))^2\Bigr)ds'}_{=1} \\
 &= \exp\Bigl(\frac{1}{2\sigma^2}(\sigma^4a^2 + 2s\sigma^2a)\Bigr) \\
 &= \exp\Bigl(\frac{\sigma^2 a^2}{2} + sa\Bigr)
\end{align*}$$
where the braced quantity is equal to 1 because it is the total integral of the probability density function of the distribution $\mathcal N(s+\sigma^2 a, \sigma^2)$.

Setting the derivative of the cost with respect to $a$ to zero, we see that the optimal action must satisfy
$$
(\sigma^2a + s)\exp(\sigma^2 a^2/2 + sa) = 0
$$
so the optimal action is $\boxed{a = -s/\sigma^2}$, which gives optimal expected cost of
$\boxed{\exp(- s^2 / (2\sigma^2))}$.

## Question 2
**Manual Value Iteration.**
Consider a simple MDP with $\mathcal{S} = \{s_1, s_2, s_3\}, \mathcal{T} =\{s_3\}, \mathcal{A} = \{a_1, a_2\}$. The State Transition Probability function
$$\mathcal{P}: \mathcal{N} \times \mathcal{A} \times \mathcal{S} \rightarrow [0, 1]$$
is defined as:
$$\mathcal{P}(s_1, a_1, s_1) = 0.2, \mathcal{P}(s_1, a_1, s_2) = 0.6, \mathcal{P}(s_1, a_1, s_3) = 0.2$$
$$\mathcal{P}(s_1, a_2, s_1) = 0.1, \mathcal{P}(s_1, a_2, s_2) = 0.2, \mathcal{P}(s_1, a_2, s_3) = 0.7$$
$$\mathcal{P}(s_2, a_1, s_1) = 0.3, \mathcal{P}(s_2, a_1, s_2) = 0.3, \mathcal{P}(s_2, a_1, s_3) = 0.4$$
$$\mathcal{P}(s_2, a_2, s_1) = 0.5, \mathcal{P}(s_2, a_2, s_2) = 0.3, \mathcal{P}(s_2, a_2, s_3) = 0.2$$
The Reward Function
$$\mathcal{R}: \mathcal{N} \times \mathcal{A} \rightarrow \mathbb{R}$$
is defined as:
$$\mathcal{R}(s_1, a_1) = 8.0, \mathcal{R}(s_1, a_2) = 10.0$$
$$\mathcal{R}(s_2, a_1) = 1.0, \mathcal{R}(s_2, a_2) = -1.0$$
Assume discount factor $\gamma = 1$.

Your task is to determine an Optimal Deterministic Policy {\em by manually working out} (not with code) simply the first two iterations of Value Iteration algorithm.

- Initialize the Value Function for each state to be it's $\max$ (over actions) reward, i.e., we initialize the Value Function to be $v_0(s_1) = 10.0, v_0(s_2) = 1.0, v_0(s_3) = 0.0$. Then manually calculate $q_k(\cdot, \cdot)$ and $v_k(\cdot)$ from $v_{k - 1}( \cdot)$ using the Value Iteration update, and then calculate the greedy policy $\pi_k(\cdot)$ from $q_k(\cdot, \cdot)$ for $k = 1$ and $k = 2$ (hence, 2 iterations).
- Now argue that $\pi_k(\cdot)$ for $k > 2$ will be the same as $\pi_2(\cdot)$. Hint: You can make the argument by examining the structure of how you get $q_k(\cdot, \cdot)$ from $v_{k-1}(\cdot)$. With this argument, there is no need to go beyond the two iterations you performed above, and so you can establish $\pi_2(\cdot)$ as an Optimal Deterministic Policy for this MDP.

We have that $q_{k+1}(s, a)$ is computed from $v_k$ by
$$\begin{array}{l|c|c}
q_{k+1}(s, a) & a_1 & a_2 \\ \hline
  s_1 & 8.0 + 0.2\cdot v_k(s_1) + 0.6\cdot v_k(s_2) &
        10 + 0.1\cdot v_k(s_1) + 0.2\cdot v_k(s_2) \\ \hline
  s_2 & 1.0 + 0.3\cdot v_k(s_1) + 0.3\cdot v_k(s_2) &
        -1.0 + 0.5\cdot v_k(s_1) + 0.3\cdot v_k(s_2)
\end{array}$$
the $k$-th policy is given by
$$\pi_k = \mathrm{argmax}_a q_k(s, a)$$
and the $k$-th value function is
$$
v_{k}(s) = \begin{cases}
  \max_a q(s, a) & \text{for } s=s_1, s_2\\
  0 &  \text{if } s=s_3
\end{cases}$$

Starting with $v_0 = (10.0, 1.0, 0.0)$, we calculate
$$\begin{align*}
q_1 &= \begin{pmatrix}
  8 + .2\cdot 10 + .6\cdot 1 & 10 + .1\cdot 10 + .2\cdot 1 \\
  1 + .3\cdot 10 + .3\cdot 1 & -1 + .5\cdot 10 + .3\cdot 1 \end{pmatrix}
  = \begin{pmatrix}
  10.6 & 11.2 \\
  4.3 & 4.3 \end{pmatrix} \\
v_1 &= (\max(10.6, 11.2), \max(4.3, 4.3), 0) = (11.2, 4.3, 0)\\
\pi_1 &= (a_2, a_1) \\ \hline
q_2 &= \begin{pmatrix}
  8 + .2\cdot 11.2 + .6\cdot 4.3 & 10 + .1\cdot 11.2 + .2\cdot 4.3 \\
  1 + .3\cdot 11.2 + .3\cdot 4.3 & -1 + .5\cdot 11.2 + .3\cdot 4.3 \end{pmatrix}
  = \begin{pmatrix}
  12.82 & 11.98 \\
  5.65 & 5.89 \end{pmatrix} \\
v_2 &= (\max(12.82, 11.98), \max(5.65, 5.89), 0) = (12.82, 5.89, 0)\\
\pi_2 &= (a_1, a_2)
\end{align*}$$

Note that since $v_0$ is non-negative, and since the reward for $a_1$ is always positive, $q_k(s, a_1)$ will always be positive, so $v_k(s_1)$ and $v_k(s_2)$ will always be positive. With this observation, we see that
$$
v_k(s_1) \ge q_k(s_1, a_2) > 10 \qquad\text{ for }k\ge 1
$$
from which it follows that
$$
v_k(s_2)\ge q_k(s_2, a_2) > -1 + .5\cdot v_{k-1}(s_1) > 4\qquad\text{ for }k\ge 2
$$
Applying these observations, we have that
$$\begin{align*}
q_{k+1}(s_1, a_1) - q_{k+1}(s_1, a_2) &= -2 + 0.1\cdot v_k(s_1) + 0.4\cdot v_k(s_2) & > 0\\
q_{k+1}(s_2, a_1) - q_{k+1}(s_2, a_2) &= 2 - 0.2\cdot v_k(s_1) & < 0
\end{align*}$$
for $k \ge 2$. This means that $\pi_k = (a_1, a_2)$ for $k \ge 2$.

In [6]:
# This cell is just checking that I didn't make an arithmetic error above.

R = np.array([[8, 10], [1, -1]])  # dimensions [s, a]
P = np.array([[[.2, .6], [.1, .2]], [[.3, .3], [.5, .3]]])  # dimensions [s, a, s']
v = np.array([10, 1])  # dimensions [s']

q = R + P @ v
v = np.max(q, axis=-1)
print(q, v)

q = R + P @ v
v = np.max(q, axis=-1)
print(q, v)

[[10.6 11.2]
 [ 4.3  4.3]] [11.2  4.3]
[[12.82 11.98]
 [ 5.65  5.89]] [12.82  5.89]


## Question 3

**Job-Hopping and Wages-Utility-Maximization.**
You are a worker who starts every day either employed or unemployed. If you start your day employed, you work on your job for the day (one of $n$ jobs, as elaborated later) and you get to earn the wage of the job for the day. However, at the end of the day, you could lose your job with probability $\alpha \in [0,1]$, in which case you start the next day unemployed. If at the end of the day, you do not lose your job (with probability $1-\alpha$), then you will start the next day with the same job (and hence, the same daily wage). On the other hand, if you start your day unemployed, then you will be randomly offered one of $n$ jobs with daily wages $w_1, w_2, \ldots w_n \in \mathbb{R}^+$ with respective job-offer probabilities $p_1, p_2, \ldots p_n \in [0,1]$ (with $\sum_{i=1}^n p_i = 1$). You can choose to either accept or decline the offered job. If you accept the job-offer, your day progresses exactly like the {\em employed-day} described above (earning the day's job wage and possibly (with probability $\alpha$) losing the job at the end of the day). However, if you decline the job-offer, you spend the day unemployed, receive the unemployment wage $w_0 \in \mathbb{R}^+$ for the day, and start the next day unemployed. The problem is to identify the optimal choice of accepting or rejecting any of the job-offers the worker receives, in a manner that maximizes the infinite-horizon {\em Expected Discounted-Sum of Wages Utility}. Assume the daily discount factor for wages (employed or unemployed) is $\gamma \in [0,1)$. Assume Wages Utility function to be $U(w) = \log(w)$ for any wage amount $w \in \mathbb{R}^+$. So you are looking to maximize
$$\mathbb{E}[\sum_{u=t}^\infty \gamma^{u-t} \cdot \log(w_{i_u})]$$
at the start of a given day $t$ ($w_{i_u}$ is the wage earned on day $u$, $0\leq i_u \leq n$ for all $u\geq t$).

- Express with clear mathematical notation the state space, action space, transition function, reward function, and write the Bellman Optimality Equation customized for this MDP.
- You can solve this Bellman Optimality Equation (hence, solve for the Optimal Value Function and the Optimal Policy) with a numerical iterative algorithm (essentially a Dynamic Programming algorithm customized to this problem). Write Python code for this numerical algorithm. Clearly define the inputs and outputs of your algorithm with their types (int, float, List, Mapping etc.). For this problem, don't use any of the MDP/DP code from the git repo, write this customized algorithm from scratch.

The state space is $\{s_1,\dots, s_n, z_1, \dots, z_n\}$, where $s_i$ represents starting the day employed in job $i$ and $z_i$ represents starting the day unemployed with an offer of job $i$. From state $s_i$, the only action is "work", and from $z_i$ the actions are "work" and "reject". The transition probabilities are
$$\begin{align*}
P(s_j|s_i, \text{work}) &= (1 - \alpha) \delta_{ij} \\
P(z_j|s_i, \text{work}) &= \alpha p_j \\
P(s_j|z_i, \text{work}) &= (1 - \alpha) \delta_{ij} \\
P(s_j|z_i, \text{reject}) &= 0 \\
P(z_j|z_i, \text{reject}) &= p_j
\end{align*}$$

The reward function is only a function of the start state and action:
$$\begin{align*}
R(s_i, \text{work}) = R(z_i, \text{work})&=  \log(w_i)\\
R(z_i, \text{reject}) &= \log(w_0)
\end{align*}$$

The Bellman optimality equation is
$$\begin{align*}
V^*(s_i) &= \log(w_i) + \gamma \Bigl[(1-\alpha)V^*(s_i) + \alpha\sum_j p_jV^*(z_j)\Bigr] \\
V^*(z_i) &= \max\Bigl(V^*(s_i),
\log(w_0) + \gamma \sum_j p_jV^*(z_j)\Bigr)
\end{align*}$$

In [7]:
@dataclasses.dataclass(frozen=True)
class WorkerPolicy:
  work: npt.NDArray[bool]  # work[i] is whether to accept an offer of job i

  def __eq__(self, other):
    return (self.work == other.work).all()


@dataclasses.dataclass(frozen=True)
class WorkerValue:
  employed: npt.NDArray[float]
  unemployed: npt.NDArray[float]


@dataclasses.dataclass(frozen=True)
class WageSolver:
  alpha: float
  wages: npt.NDArray[float]
  offer_probabilities: npt.NDArray[float]

  def compute_value(self, gamma: float, policy: WorkerPolicy) -> WorkerValue:
    """Value function for a given policy."""
    # This could be done by iterating the Bellman operator to convergence, but
    # here we solve the Bellman linear system for V^pi with np.linalg.solve.
    reward = np.log(self.wages)
    R = np.concatenate([reward[1:],
                        np.where(policy.work, reward[1:], reward[0])])
    # Transition matrix if you're employed, ignoring job loss.
    job = np.concatenate([np.eye(n), np.zeros((n, n))], axis=1)
    # Transition matrix if you're unemployed and reject work.
    no_job = np.concatenate([np.zeros((n, n)), np.tile(self.offer_probabilities, (n, 1))], axis=1)
    # Transition matrix if you're employed, taking possible job loss into account.
    job = (1 - self.alpha) * job + self.alpha * no_job
    # Transition matrix if you're unemployed and follow the policy.
    no_job = np.where(policy.work[:, np.newaxis], job, no_job)
    # Full transition matrix.
    P = np.concatenate([job, no_job], axis=0)
    V = np.linalg.solve(np.eye(2 * n) - gamma * P, R)
    return WorkerValue(V[:n], V[n:])

  def compute_greedy_policy(self, gamma: float, value: WorkerValue) -> WorkerPolicy:
    value_work = value.employed
    value_reject = np.log(self.wages[0]) + gamma * (self.offer_probabilities @ value.unemployed)
    return WorkerPolicy(value_work >= value_reject)

  def iterate_policy_and_value(
      self,
      gamma: float,
      initial_policy: Optional[WorkerPolicy] = None
      ) -> Iterator[tuple[WorkerPolicy, WorkerValue]]:
    """Yields policies and their values. Next policy is greedy wrt previous value."""
    if initial_policy is not None:
      policy = initial_policy
    else:
      policy = WorkerPolicy(np.array([True for _ in self.offer_probabilities]))

    while True:
      value = self.compute_value(gamma, policy)
      yield policy, value
      new_policy = self.compute_greedy_policy(gamma, value)
      if new_policy == policy:
        break
      policy = new_policy

  def optimal_policy_and_value(self, gamma: float) -> tuple[WorkerPolicy, WorkerValue]:
    for policy, value in self.iterate_policy_and_value(gamma):
      pass
    return policy, value

In [8]:
# Some choice of alpha, w_i, and p_i, for demonstration purposes.

rng = np.random.default_rng(1)
n = 6
alpha = 0.1
wages = 10 * rng.random(n + 1)
print(f'Unemployment wage: {wages[0]}\nWages for jobs:\n{wages[1:]}')
offer_probabilities = rng.random(n)
offer_probabilities = offer_probabilities / offer_probabilities.sum()
print(f'Offer probabilities for jobs:\n{offer_probabilities}')
solver = WageSolver(0.1, wages, offer_probabilities)

Unemployment wage: 5.118216247002567
Wages for jobs:
[9.50463696 1.44159613 9.48649447 3.11831452 4.23326449 8.27702594]
Offer probabilities for jobs:
[0.15691715 0.21075478 0.0105682  0.28895254 0.20636386 0.12644347]


In [9]:
# Solve the MDP for a given gamma.
gamma = 0.99
for policy, value in solver.iterate_policy_and_value(gamma):
  print(f'{policy}\n{value}')

WorkerPolicy(work=array([ True,  True,  True,  True,  True,  True]))
WorkerValue(employed=array([143.07712264, 125.77410558, 143.05959394, 132.85246996,
       135.65688101, 141.80835142]), unemployed=array([143.07712264, 125.77410558, 143.05959394, 132.85246996,
       135.65688101, 141.80835142]))
WorkerPolicy(work=array([ True, False,  True, False,  True,  True]))
WorkerValue(employed=array([189.46570846, 172.1626914 , 189.44817976, 179.24105578,
       182.04546683, 188.19693724]), unemployed=array([189.46570846, 185.6326304 , 189.44817976, 185.6326304 ,
       182.04546683, 188.19693724]))
WorkerPolicy(work=array([ True, False,  True, False, False,  True]))
WorkerValue(employed=array([209.22512973, 191.92211267, 209.20760103, 199.00047705,
       201.8048881 , 207.95635851]), unemployed=array([209.22512973, 207.17039959, 209.20760103, 207.17039959,
       207.17039959, 207.95635851]))


## Scrap

Just for fun, how would we solve this problem with the framework from the RL-book repo?

In [10]:
@dataclasses.dataclass(frozen=True)
class WorkerState:
  job_id: int
  offer: int = 0  # Only set if job_id is 0.

  def unemployed(self):
    return self.job_id == 0


def maybe_lose_job(alpha: float):
  def _lose_job(state: WorkerState) -> FiniteDistribution[WorkerState]:
    if state.unemployed():
      return Constant(state)
    return Categorical({WorkerState(0): alpha, state: 1 - alpha})
  return _lose_job


def get_offer_if_unemployed(offer_probabilities: list[float]):
  def _get_offer(state: WorkerState) -> FiniteDistribution[WorkerState]:
    if state.unemployed():
      return Categorical({dataclasses.replace(state, offer=i+1): p for i, p in enumerate(offer_probabilities)})
    return Constant(state)
  return _get_offer


A = TypeVar('A')
B = TypeVar('B')

def apply(self: FiniteDistribution[A], f: Callable[[A], FiniteDistribution[B]]) -> FiniteDistribution[B]:
  result: Dict[B, float] = collections.defaultdict(float)
  for x, p in self:
    for y, q in f(x):
      result[y] += p * q
  return Categorical(result)

# This is not quite appropriate, since the signature of the base class is
# violated when `f` does not return a FiniteDistribution. :-/
FiniteDistribution.apply = apply


def worker_mapping(wages: list[float], offer_probabilities: list[float], alpha: float):
  """State-action mapping object for given parameters."""
  n = len(offer_probabilities)
  assert len(wages) == n + 1  # wages[0] being the unemployment wage
  reward = [np.log(w) for w in wages]

  def do_job(i: int) -> FiniteDistribution[tuple[WorkerState, float]]:
    """State-reward distribution if you do work i for the day."""
    return Constant(WorkerState(i)
        ).apply(maybe_lose_job(alpha)
        ).apply(get_offer_if_unemployed(offer_probabilities)
        ).map(lambda s: (s, reward[i]))

  # If you're employed (i.e. not in state 0), your only available action is to
  # work.
  mapping = {WorkerState(i): {'work': do_job(i)} for i in range(1, n + 1)}
  # If you're unemployed, you can either accept or reject the offered job.
  mapping.update({
      WorkerState(0, i): {'work': do_job(i), 'reject': do_job(0)}
      for i in range(1, n + 1)
  })
  return mapping

class WorkerMDP(FiniteMarkovDecisionProcess[WorkerState, str]):
  def __init__(self, wages: list[float], offer_probabilities: list[float], alpha: float):
    super().__init__(worker_mapping(wages, offer_probabilities, alpha))

In [11]:
from rl.iterate import iterate
from rl.policy import FiniteDeterministicPolicy
import operator
S = TypeVar('S')
V = Mapping[NonTerminal[S], float]

def extended_vf(v: V[S], s: State[S]) -> float:
    def non_terminal_vf(st: NonTerminal[S], v=v) -> float:
        return v[st]
    return s.on_non_terminal(non_terminal_vf, 0.0)

def greedy_policy_from_vf(
    mdp: FiniteMarkovDecisionProcess[S, A],
    vf: V[S],
    gamma: float
) -> FiniteDeterministicPolicy[S, A]:
    greedy_policy_dict: Dict[S, A] = {}

    for s in mdp.non_terminal_states:
        q_values: Iterator[tuple[A, float]] = \
            ((a, mdp.mapping[s][a].expectation(
                lambda s_r: s_r[1] + gamma * extended_vf(vf, s_r[0])
            )) for a in mdp.actions(s))
        greedy_policy_dict[s.state] = \
            max(q_values, key=operator.itemgetter(1))[0]

    return FiniteDeterministicPolicy(greedy_policy_dict)

def policy_iteration(
    mdp: FiniteMarkovDecisionProcess[S, A],
    gamma: float,
) -> Iterator[tuple[V[S], FinitePolicy[S, A]]]:

    def update(vf_policy: tuple[V[S], FinitePolicy[S, A]])\
            -> tuple[V[S], FiniteDeterministicPolicy[S, A]]:

        vf, pi = vf_policy
        improved_pi: FiniteDeterministicPolicy[S, A] = greedy_policy_from_vf(
            mdp, vf, gamma)
        mrp: FiniteMarkovRewardProcess[S] = mdp.apply_finite_policy(improved_pi)
        improved_vf: V[S] = dict(zip(mrp.non_terminal_states, mrp.get_value_function_vec(gamma)))

        return improved_vf, improved_pi

    pi_0 = FiniteDeterministicPolicy(
        {s.state: next(iter(mdp.actions(s))) for s in mdp.non_terminal_states}
    )
    mrp = mdp.apply_finite_policy(pi_0)
    v_0 = dict(zip(mrp.non_terminal_states, mrp.get_value_function_vec(gamma)))
    return iterate(update, (v_0, pi_0))

In [12]:
mdp = WorkerMDP(wages, offer_probabilities, alpha)

In [13]:
prev_policy = None
for value, policy in policy_iteration(mdp, gamma):
  if policy == prev_policy:
    break
  prev_policy = policy
  for s in mdp.non_terminal_states:
    print(f'{s}: action {policy.action_for[s.state]}, value {value[s]}')
  print('-' * 20)


NonTerminal(state=WorkerState(job_id=1, offer=0)): action work, value 143.07712264303981
NonTerminal(state=WorkerState(job_id=2, offer=0)): action work, value 125.77410558328585
NonTerminal(state=WorkerState(job_id=3, offer=0)): action work, value 143.05959394224564
NonTerminal(state=WorkerState(job_id=4, offer=0)): action work, value 132.85246995511804
NonTerminal(state=WorkerState(job_id=5, offer=0)): action work, value 135.65688100519782
NonTerminal(state=WorkerState(job_id=6, offer=0)): action work, value 141.80835142054144
NonTerminal(state=WorkerState(job_id=0, offer=1)): action work, value 143.07712264303981
NonTerminal(state=WorkerState(job_id=0, offer=2)): action work, value 125.77410558328586
NonTerminal(state=WorkerState(job_id=0, offer=3)): action work, value 143.05959394224564
NonTerminal(state=WorkerState(job_id=0, offer=4)): action work, value 132.85246995511807
NonTerminal(state=WorkerState(job_id=0, offer=5)): action work, value 135.65688100519782
NonTerminal(state=Wor

## Question 4

**Two-Stores Inventory Control.**
We extend the capacity-constrained inventory example implemented in [rl/chapter3/simple_inventory_mdp_cap.py](https://github.com/TikhonJelvis/RL-book/blob/master/rl/chapter3/simple_inventory_mdp_cap.py) as a `FiniteMarkovDecisionProcess` (the Finite MDP model for the capacity-constrained inventory example is described in detail in Chapters 1 and 2 of the RLForFinanceBook). Here we assume that we have two different stores, each with their own separate capacities $C_1$ and $C_2$, their own separate Poisson probability distributions of demand (with means $\lambda_1$ and $\lambda_2$), their own separate holding costs $h_1$ and $h_2$, and their own separate stockout costs $p_1$ and $p_2$. At 6pm upon stores closing each evening, each store can choose to order inventory from a common supplier (as usual, ordered inventory will arrive at the store 36 hours later). We are also allowed to transfer inventory from one store to another, and any such transfer happens overnight, i.e., will arrive by 6am next morning (since the stores are fairly close to each other). Note that the orders are constrained such that following the orders on each evening, each store's inventory position (sum of on-hand inventory and on-order inventory) cannot exceed the store's capacity (this means the action space is constrained to be finite). Each order made to the supplier incurs a fixed transportation cost of $K_1$ (fixed-cost means the cost is the same no matter how many units of non-zero inventory a particular store orders). Moving any non-zero inventory between the two stores incurs a fixed transportation cost of $K_2$.

Model this as a derived class of `FiniteMarkovDecisionProcess` much like we did for `SimpleInventoryMDPCap` in the code repo. Set up instances of this derived class for different choices of the problem parameters (capacities, costs etc.), and determine the Optimal Value Function and Optimal Policy by invoking the function `value_iteration` (or `policy_iteration`) from file [rl/dynamic_programming.py](https://github.com/TikhonJelvis/RL-book/blob/master/rl/dynamic_programming.py).

Analyze the obtained Optimal Policy and verify that it makes intuitive sense as a function of the problem parameters.

In [47]:
@dataclasses.dataclass(frozen=True)
class StoreState:
  """State of a single store."""
  on_hand: int
  on_order: int

  def inventory_position(self) -> int:
    return self.on_hand + self.on_order


@dataclasses.dataclass(frozen=True)
class Store:
  """Parameters of a single store."""
  capacity: int
  poisson_lambda: float
  holding_cost: float
  stockout_cost: float

  def available_space(self, state) -> int:
    return self.capacity - state.inventory_position()

  def possible_states(self) -> Iterator[StoreState]:
    for on_hand in range(self.capacity + 1):
      for on_order in range(self.capacity + 1 - on_hand):
        yield StoreState(on_hand, on_order)

  def state_reward_distr(self, state: StoreState, order: int, immediate_additional_on_hand: int) -> FiniteDistribution[tuple[StoreState, float]]:
    """Distribution of next states and holding+stockout costs for this store,
    assuming a given action.

    Args:
      state: the starting store state.
      order: inventory this store orders from the supplier.
      immediate_additional_on_hand: number of units ordered from the other store
         If positive, this inventory is immediately added to our inventory, but
         for we don't pay holding cost on it.
         If negative, this inventory is immediately subtracted from our
         inventory, and we don't pay holding cost on it.
    """
    # We use the same expected reward trick used in
    # rl/chapter3/simple_inventory_mdp_cap.py to make the distribution finite.
    ip = state.inventory_position() + immediate_additional_on_hand
    if immediate_additional_on_hand >= 0:
      # When receiving extra inventory, you pay holding cost for what you had on
      # hand the previous night.
      base_reward = - self.holding_cost * state.on_hand
    else:
      # When sending inventory to another store, you pay holding cost for what
      # you had on hand the previous night _minus_ what you sent out that night.
      base_reward = - self.holding_cost * (state.on_hand + immediate_additional_on_hand)
    poisson_distr = scipy.stats.poisson(self.poisson_lambda)

    sr_probs_dict = {
        (StoreState(ip - i, order), base_reward): poisson_distr.pmf(i)
        for i in range(ip)}

    probability = 1 - poisson_distr.cdf(ip - 1)
    reward = base_reward - self.stockout_cost * (
        self.poisson_lambda - ip * (1 - poisson_distr.pmf(ip) / probability))
    sr_probs_dict[(StoreState(0, order), reward)] = probability
    return Categorical(sr_probs_dict)


@dataclasses.dataclass(frozen=True)
class TwoStoreState:
  store1: StoreState
  store2: StoreState


@dataclasses.dataclass(frozen=True)
class TwoStoreAction:
  order1: int
  order2: int
  move_from_1_to_2: int  # Can be negative to indicate movement from 2 to 1.

TwoStoreMapping = Mapping[
    TwoStoreState,
    Mapping[TwoStoreAction, Categorical[tuple[TwoStoreState, float]]]
]

@dataclasses.dataclass(frozen=True)
class TwoStores:
  """Parameters of the two-store system."""
  store1: Store
  store2: Store
  supplier_cost: float
  inter_store_cost: float

  def possible_states(self) -> Iterator[TwoStoreState]:
    for state1 in self.store1.possible_states():
      for state2 in self.store2.possible_states():
        yield TwoStoreState(state1, state2)

  def action_cost(self, action: TwoStoreAction) -> float:
    """The inventory-moving cost of an action. Does not include individual store
    holding costs or stockout costs."""
    return (self.inter_store_cost * int(action.move_from_1_to_2 != 0) +
            self.supplier_cost * (int(action.order1 != 0) + int(action.order2 != 0)))

  def possible_actions(self, state: TwoStoreState) -> Iterator[TwoStoreAction]:
    as1 = self.store1.available_space(state.store1)
    as2 = self.store2.available_space(state.store2)
    max_2_to_1 = min(as1, state.store2.on_hand)
    max_1_to_2 = min(as2, state.store1.on_hand)
    for move_from_1_to_2 in range(-max_2_to_1, max_1_to_2 + 1):
      for order1 in range(as1 + move_from_1_to_2 + 1):
        for order2 in range(as2 - move_from_1_to_2 + 1):
          max_2_to_1 = min(as1 - order1, state.store2.on_hand)
          max_1_to_2 = min(as2 - order2, state.store1.on_hand)
          for move_from_1_to_2 in range(-max_2_to_1, max_1_to_2 + 1):
            yield TwoStoreAction(order1, order2, move_from_1_to_2)

  def get_action_transition_reward_map(self) -> TwoStoreMapping:
    d: TwoStoreMapping = {}
    for state in self.possible_states():
      d1 = collections.defaultdict(float)
      for action in self.possible_actions(state):
        base_reward = -self.action_cost(action)
        d2 = collections.defaultdict(float)
        for (next_state1, reward1), prob1 in self.store1.state_reward_distr(state.store1, action.order1, -action.move_from_1_to_2):
          for (next_state2, reward2), prob2 in self.store2.state_reward_distr(state.store2, action.order2, action.move_from_1_to_2):
            reward = base_reward + reward1 + reward2
            d2[(TwoStoreState(next_state1, next_state2), reward)] += prob1 * prob2
        d1[action] = Categorical(d2)
      d[state] = d1
    return d

class TwoStoreMDP(FiniteMarkovDecisionProcess[TwoStoreState, int]):
  def __init__(self, stores: TwoStores):
    super().__init__(stores.get_action_transition_reward_map())

In [76]:
stores = TwoStores(Store(capacity=2, poisson_lambda=1.3, holding_cost=.16, stockout_cost=5.1),
                   Store(capacity=3, poisson_lambda=2.4, holding_cost=.11, stockout_cost=4.9),
                   supplier_cost=1.0,
                   inter_store_cost=0.2)

In [77]:
# Check that possible actions are as expected.
list(stores.possible_actions(TwoStoreState(store1=StoreState(on_hand=1, on_order=0),
                                           store2=StoreState(on_hand=2, on_order=1))))

[TwoStoreAction(order1=0, order2=0, move_from_1_to_2=-1),
 TwoStoreAction(order1=0, order2=0, move_from_1_to_2=0),
 TwoStoreAction(order1=0, order2=1, move_from_1_to_2=-1),
 TwoStoreAction(order1=0, order2=0, move_from_1_to_2=-1),
 TwoStoreAction(order1=0, order2=0, move_from_1_to_2=0),
 TwoStoreAction(order1=1, order2=0, move_from_1_to_2=0)]

In [78]:
mdp = TwoStoreMDP(stores)
mdp

From State TwoStoreState(store1=StoreState(on_hand=0, on_order=0), store2=StoreState(on_hand=0, on_order=0)):
  With Action TwoStoreAction(order1=0, order2=0, move_from_1_to_2=0):
    To [State TwoStoreState(store1=StoreState(on_hand=0, on_order=0), store2=StoreState(on_hand=0, on_order=0)) and Reward -18.390] with Probability 1.000
  With Action TwoStoreAction(order1=0, order2=1, move_from_1_to_2=0):
    To [State TwoStoreState(store1=StoreState(on_hand=0, on_order=0), store2=StoreState(on_hand=0, on_order=1)) and Reward -19.390] with Probability 1.000
  With Action TwoStoreAction(order1=0, order2=2, move_from_1_to_2=0):
    To [State TwoStoreState(store1=StoreState(on_hand=0, on_order=0), store2=StoreState(on_hand=0, on_order=2)) and Reward -19.390] with Probability 1.000
  With Action TwoStoreAction(order1=0, order2=3, move_from_1_to_2=0):
    To [State TwoStoreState(store1=StoreState(on_hand=0, on_order=0), store2=StoreState(on_hand=0, on_order=3)) and Reward -19.390] with Probabil

In [69]:
# Optimal policy and value for one example.
stores = TwoStores(Store(capacity=2, poisson_lambda=1.3, holding_cost=.16, stockout_cost=5.1),
                   Store(capacity=3, poisson_lambda=2.4, holding_cost=.11, stockout_cost=4.9),
                   supplier_cost=1.0,
                   inter_store_cost=0.2)
mdp = TwoStoreMDP(stores)
opt_vf, opt_policy = rl.dynamic_programming.value_iteration_result(mdp, gamma=.99)
for s, a in opt_policy.action_for.items():
  print(f'{s.store1, s.store2} -> {a}, value {opt_vf[NonTerminal(s)]}')

(StoreState(on_hand=0, on_order=0), StoreState(on_hand=0, on_order=0)) -> TwoStoreAction(order1=2, order2=3, move_from_1_to_2=0), value -963.478061135524
(StoreState(on_hand=0, on_order=0), StoreState(on_hand=0, on_order=1)) -> TwoStoreAction(order1=2, order2=2, move_from_1_to_2=0), value -960.8849829571728
(StoreState(on_hand=0, on_order=0), StoreState(on_hand=0, on_order=2)) -> TwoStoreAction(order1=2, order2=1, move_from_1_to_2=0), value -958.9042939718491
(StoreState(on_hand=0, on_order=0), StoreState(on_hand=0, on_order=3)) -> TwoStoreAction(order1=2, order2=0, move_from_1_to_2=0), value -956.9957630445618
(StoreState(on_hand=0, on_order=0), StoreState(on_hand=1, on_order=0)) -> TwoStoreAction(order1=2, order2=2, move_from_1_to_2=0), value -960.9949829571729
(StoreState(on_hand=0, on_order=0), StoreState(on_hand=1, on_order=1)) -> TwoStoreAction(order1=1, order2=2, move_from_1_to_2=-1), value -958.8647689155023
(StoreState(on_hand=0, on_order=0), StoreState(on_hand=1, on_order=2))

In [75]:
# If one store has no capacity (but also no stockout cost, or no customers),
# then the optimal policies and actions should match the 1-store example.
stores = TwoStores(Store(capacity=2, poisson_lambda=1.0, holding_cost=1.0, stockout_cost=10.0),
                   Store(capacity=0, poisson_lambda=0.0, holding_cost=0.0, stockout_cost=0.0),
                   supplier_cost=0.0,
                   inter_store_cost=0.0)
mdp = TwoStoreMDP(stores)
opt_vf, opt_policy = rl.dynamic_programming.value_iteration_result(mdp, gamma=.9)
for s, a in opt_policy.action_for.items():
  print(f'{s.store1, s.store2} -> {a}, value {opt_vf[NonTerminal(s)]}')

(StoreState(on_hand=0, on_order=0), StoreState(on_hand=0, on_order=0)) -> TwoStoreAction(order1=2, order2=0, move_from_1_to_2=0), value -43.59563313047815
(StoreState(on_hand=0, on_order=1), StoreState(on_hand=0, on_order=0)) -> TwoStoreAction(order1=1, order2=0, move_from_1_to_2=0), value -37.97111179441265
(StoreState(on_hand=0, on_order=2), StoreState(on_hand=0, on_order=0)) -> TwoStoreAction(order1=0, order2=0, move_from_1_to_2=0), value -37.3284904356655
(StoreState(on_hand=1, on_order=0), StoreState(on_hand=0, on_order=0)) -> TwoStoreAction(order1=1, order2=0, move_from_1_to_2=0), value -38.97111179441265
(StoreState(on_hand=1, on_order=1), StoreState(on_hand=0, on_order=0)) -> TwoStoreAction(order1=0, order2=0, move_from_1_to_2=0), value -38.3284904356655
(StoreState(on_hand=2, on_order=0), StoreState(on_hand=0, on_order=0)) -> TwoStoreAction(order1=0, order2=0, move_from_1_to_2=0), value -39.3284904356655


In [86]:
# If hardly anyone ever comes to the store, it's never worth ordering inventory.
stores = TwoStores(Store(capacity=2, poisson_lambda=0.01, holding_cost=1.0, stockout_cost=10.0),
                   Store(capacity=3, poisson_lambda=0.01, holding_cost=1.1, stockout_cost=11.0),
                   supplier_cost=1.0,
                   inter_store_cost=0.2)
mdp = TwoStoreMDP(stores)
opt_vf, opt_policy = rl.dynamic_programming.value_iteration_result(mdp, gamma=.9)
for s, a in opt_policy.action_for.items():
  assert a.order1 == 0 and a.order2 == 0